In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load the modified price data and slice according to the public period
public = pd.read_csv('/content/drive/MyDrive/학교/종목 추천/good.csv', encoding = "cp949")
public['Date'] = pd.to_datetime(public['Date'], format='%Y-%m-%d')
public.set_index('Date', inplace=True)
public = public.loc[:'2023-11-01']  # Update the date range to the new end date

In [4]:
# Load the modified price data for the private period
private = pd.read_csv('/content/drive/MyDrive/학교/종목 추천/good.csv', encoding='cp949')
private['Date'] = pd.to_datetime(private['Date'], format='%Y-%m-%d')
private.set_index('Date', inplace=True)

In [5]:
# Extract the unique stock codes from the training data
unique_codes = public['Code'].unique()

In [8]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# 데이터 파일을 읽어들입니다.
kospi_data = pd.read_csv('/content/drive/MyDrive/학교/종목 추천/kospi_data.csv')
stock_data = pd.read_csv('/content/drive/MyDrive/학교/종목 추천/good.csv')

# 날짜 형식을 datetime으로 변환합니다.
kospi_data['Date'] = pd.to_datetime(kospi_data['Date'])
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# 수익률을 계산하는 함수를 정의합니다.
def calculate_returns(df):
    df['Return'] = df['Close'].pct_change()  # pct_change() 함수를 사용하여 수익률을 계산합니다.
    return df

# 시장 수익률을 계산합니다.
kospi_data = calculate_returns(kospi_data)

# 시장 수익률의 분산을 계산합니다.
market_variance = kospi_data['Return'].var()

# 베타계수를 저장할 딕셔너리를 준비합니다.
betas = {}

# 개별 종목 데이터를 처리합니다.
for code in tqdm(stock_data['Code'].unique(), desc='Calculating Betas'):
    # 특정 종목의 데이터만 선택합니다.
    stock = stock_data[stock_data['Code'] == code]
    stock = calculate_returns(stock)

    # 시장 데이터와 종목 데이터를 날짜 기준으로 결합합니다.
    merged_data = pd.merge(stock, kospi_data, on='Date', suffixes=('_stock', '_market'), how='inner')

    # 공분산을 계산합니다.
    covariance = np.cov(merged_data['Return_stock'].dropna(), merged_data['Return_market'].dropna())[0, 1]

    # 베타계수를 계산하고 저장합니다.
    beta = covariance / market_variance
    betas[code] = beta

# 베타 계수 결과를 데이터 프레임으로 변환합니다.
betas_df = pd.DataFrame(betas.items(), columns=['Code', 'Beta'])

Calculating Betas:   0%|          | 0/2000 [00:00<?, ?it/s]

In [9]:
# 이 부분은 tqdm과 경고 필터링을 설정하는 부분입니다.
from tqdm.auto import tqdm
warnings.filterwarnings("ignore")

# 결과를 저장할 DataFrame을 생성합니다.
results_df = pd.DataFrame()

# 15일부터 60일까지 5일 간격으로 윈도우 사이즈를 변경하면서 계산을 반복합니다.
for window_size in range(15, 65, 5):
    window_results = []
    # tqdm을 사용하여 진행률을 표시합니다.
    for code in tqdm(unique_codes, desc=f"Window Size {window_size}"):
        # 해당 종목 코드의 종가와 변동률을 가져옵니다.
        public_close = public[public['Code'] == code][['Close','Change']]
        # 윈도우 사이즈에 해당하는 마지막 데이터를 가져옵니다.
        public_close_window = public_close[-window_size:]
        # 해당 기간 동안의 최종 수익률을 계산합니다.
        final_return = (public_close_window['Close'].iloc[-1] - public_close_window['Close'].iloc[0]) / public_close_window['Close'].iloc[0]
        # 해당 기간 동안의 일일 변동률의 표준편차를 계산합니다.
        std_deviation = np.std(public_close_window['Change'])
        # 샤프 비율을 계산합니다.
        sharpe_ratio = final_return/std_deviation if std_deviation != 0 else 0
        # 결과를 리스트에 추가합니다.
        window_results.append({'종목코드': code, 'Sharpe': sharpe_ratio, 'Window Size': window_size})

    # 결과 리스트를 DataFrame으로 변환하여 결과 DataFrame에 추가합니다.
    results_df = results_df.append(window_results, ignore_index=True)

# 결과 DataFrame을 출력합니다.
results_df.head()

Window Size 15:   0%|          | 0/2000 [00:00<?, ?it/s]

Window Size 20:   0%|          | 0/2000 [00:00<?, ?it/s]

Window Size 25:   0%|          | 0/2000 [00:00<?, ?it/s]

Window Size 30:   0%|          | 0/2000 [00:00<?, ?it/s]

Window Size 35:   0%|          | 0/2000 [00:00<?, ?it/s]

...

Window Size 45:   0%|          | 0/2000 [00:00<?, ?it/s]

Window Size 50:   0%|          | 0/2000 [00:00<?, ?it/s]

Window Size 55:   0%|          | 0/2000 [00:00<?, ?it/s]

Window Size 60:   0%|          | 0/2000 [00:00<?, ?it/s]

,종목코드,Sharpe,Window Size
0,A005930,-0.301784,15
1,A000660,-1.146565,15
2,A207940,-1.923322,15
3,A006400,-5.584008,15
4,A051910,-6.314527,15


In [17]:
# 결과 DataFrame에 순위를 부여합니다.
results_df['순위'] = results_df.groupby('Window Size')['Sharpe'].rank(method='first', ascending=False)

# sample_submission.csv 파일을 읽어옵니다.
sample_submission = pd.read_csv('/content/drive/MyDrive/학교/종목 추천/sample_submission.csv')

# 샤프 비율 순위에 따라 종목코드를 병합합니다.
for window_size in results_df['Window Size'].unique():
    window_rank = results_df[results_df['Window Size'] == window_size][['종목코드', '순위']]
    merged_rank = sample_submission.merge(window_rank, on='종목코드', how='left')
    # 병합된 순위를 파일로 저장합니다.
    merged_rank.to_csv(f'/content/drive/MyDrive/학교/종목 추천/submission_window_{window_size}.csv', index=False)

# private DataFrame의 인덱스 중복을 제거합니다.
private = private[~private.index.duplicated(keep='last')]

validation_start_date = '2023-10-01'
validation_end_date = '2023-10-31'
public_test = private.loc[validation_start_date:validation_end_date]

In [18]:
def calculate_return_for_window(window_size):
    # 윈도우 사이즈별 순위 데이터를 로드합니다.
    window_rank_df = pd.read_csv(f'/content/drive/MyDrive/학교/종목 추천/submission_window_{window_size}.csv')

    # 파일에 '순위' 컬럼이 있는지 확인하고, 없으면 적절한 컬럼으로 변경합니다.
    if '순위' not in window_rank_df.columns:
        if '순위_y' in window_rank_df.columns:
            window_rank_df = window_rank_df.rename(columns={'순위_y': '순위'})
        else:
            # '순위' 컬럼이 없다면 오류를 출력하거나 기본 값을 설정합니다.
            print(f"'순위' 컬럼이 '{window_size}' 윈도우 사이즈 파일에 없습니다.")
            return None  # 또는 적절한 기본값을 반환할 수 있습니다.
    # 상위 순위 종목코드 리스트
    top_codes = window_rank_df.sort_values(by='순위').head(200)['종목코드'].values

    # 검증 기간 동안 상위 종목의 종가와 코드를 가져옵니다.
    top_prices = public_test[public_test['Code'].isin(top_codes)][['Code', 'Close']]

    # 'Code' 열이 있는지 확인합니다.
    if 'Code' not in top_prices.columns:
        print(f"'Code' 열이 없습니다.")
        return None

    # 검증 기간 시작 종가와 종료 종가를 비교하여 수익률을 계산합니다.
    start_prices = top_prices.groupby('Code')['Close'].nth(0)  # 검증 기간 시작 종가
    end_prices = top_prices.groupby('Code')['Close'].nth(-1)   # 검증 기간 종료 종가
    returns = (end_prices - start_prices) / start_prices  # 수익률 계산

    # 평균 수익률을 반환합니다.
    return returns.mean()

# 각 윈도우 사이즈별로 수익률을 계산하고 결과를 저장합니다.
window_returns = {}
for window_size in range(15, 65, 5):
    window_returns[window_size] = calculate_return_for_window(window_size)

# 가장 높은 평균 수익률을 가진 윈도우 사이즈를 찾습니다.
best_window = max(window_returns, key=window_returns.get)
print(f"Best window size: {best_window} with an average return of {window_returns[best_window]}")

Best window size: 15 with an average return of 0.22774193548387098


In [19]:
# 최적의 윈도우 사이즈 설정
optimal_window_size = 15

# 결과를 저장할 데이터프레임 초기화
optimal_results_df = pd.DataFrame()

# 최적의 윈도우 사이즈를 사용하여 각 종목에 대해 샤프 비율을 계산합니다.
for code in tqdm(unique_codes, desc=f"Calculating Sharpe Ratios for Window Size {optimal_window_size}"):
    # 해당 종목 코드의 종가와 변동률을 가져옵니다.
    public_close = public[public['Code'] == code][['Close', 'Change']]
    # 윈도우 사이즈에 해당하는 데이터를 가져옵니다.
    public_close_window = public_close[-optimal_window_size:]
    # 해당 기간 동안의 최종 수익률을 계산합니다.
    final_return = (public_close_window['Close'].iloc[-1] - public_close_window['Close'].iloc[0]) / public_close_window['Close'].iloc[0]
    # 해당 기간 동안의 일일 변동률의 표준편차를 계산합니다.
    std_deviation = np.std(public_close_window['Change'])
    # 샤프 비율을 계산합니다.
    sharpe_ratio = final_return / std_deviation if std_deviation != 0 else 0
    # 결과를 데이터프레임에 추가합니다.
    optimal_results_df = optimal_results_df.append({'종목코드': code, 'Sharpe': sharpe_ratio}, ignore_index=True)

# 베타 계수를 계산합니다. (이전에 베타 계수를 계산하는 코드를 실행했다고 가정)

# 베타 계수 데이터프레임과 샤프 비율 데이터프레임을 결합합니다.
final_df = pd.merge(optimal_results_df, betas_df, on='종목코드')

# 최종 데이터프레임에서 종목을 평가하고 선택합니다.
# 예를 들어, 샤프 비율이 높고 베타 계수가 낮은 종목을 선호할 수 있습니다.
selected_stocks = final_df.sort_values(by=['Sharpe', 'Beta'], ascending=[False, True]).head(5)

# 선택된 종목을 출력합니다.
print(selected_stocks)

Calculating Sharpe Ratios for Window Size 15:   0%|          | 0/2000 [00:00<?, ?it/s]

         종목코드    Sharpe      Beta
305   A268280  9.069213 -0.021030
443   A014830  8.594681  0.969867
1564  A064520  7.691134  0.263285
213   A000240  7.025759  0.573388
665   A089030  6.890244  1.051980
